<h1 align="center">An Introduction to Graph Databases with GQLAlchemy and Python</h1>

# Table of contents <a name="toc"></a>
1. [Introduction](#introduction)
2. [Prerequisites](#prerequisites)
    1. [Memgraph installation using Docker](#memgraph-docker)
3. [Dataset and graph model](#dataset)
4. [Connect to Memgraph with GQLAlchemy](#connect)
5. [Define a graph schema](#graph-schema)
6. [Creating and returning nodes and relationships](#create-return)
7. [Importing data from CSV files](#import)
8. [Querying the database and retrieving results](#query)
9. [Example queries](#exercises)
10. [Want to know more?](#extra)
    1. [Movie recommender](#movie)
    2. [Graph analytics](#graph-analytics)

## 1. Introduction <a name="introduction"></a>

Through this tutorial, we will learn how to create a graph model from a dataset, run Memgraph with Docker, connect to it from a Jupyter Notebook with the help of GQLAlchemy, and perform simple queries. We will explore the **movies dataset** consisting of two CSV files: `movies.csv` containing movie title and genre and `ratings.csv` that holds information about users' ratings.


## 2. Prerequisites <a name="prerequisites"></a>

For this tutorial, we need to install:

- [Jupyter](https://jupyter.org/install) - to run the Jupyter notebook
- [Docker](https://docs.docker.com/get-docker/) - to run Memgraph, since Memgraph is a native Linux application and cannot be installed on Windows and macOS
- [GQLAlchemy](https://pypi.org/project/gqlalchemy/) - to connect to Memgraph and perform queries
- [Pandas](https://pypi.org/project/pandas/) - to show how the CSV files look like

### A. Memgraph installation using Docker <a name="memgraph-docker"></a>

After we install Docker, we can set up **Memgraph** by running:

```
docker run -it -p 7687:7687 -p 3000:3000 memgraph/memgraph-platform:2.2.5
```

This command will start the download and after it finishes, run the Memgraph container.

**Memgraph Platform** contains:

- **Memgraph DB** - the database that holds your data
- **Memgraph Lab** - visual user interface for running queries and visualizing graph data (running at `localhost:3000`)
- **mgconsole** - command-line interface for running queries
- **MAGE** - graph algorithms and modules library

## 3. Dataset and graph model <a name="dataset"></a>

We will use the Pandas library to show how `movies.csv` and `ratings.csv` files look like.

In [1]:
import pandas as pd

movies = pd.read_csv("../data/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
ratings = pd.read_csv("../data/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


*** 

Next, we must create a **graph model** for the movies dataset. Every movie has a title and genre it belongs to. Therefore, let's create `:Movie` and `:Genre` nodes and the relationship between them of `OF_GENRE` type. The `:Movie` nodes will have `id` and `title` properties, while `:Genre` nodes will have just `name` property. In `ratings.csv`, we have users who rated movies at a certain moment. Hence, we should create `:User` nodes and a new relationship type called `RATED` between `:User` and `:Movie` nodes. This relationship will have properties `rating` and `timestamp`. `:User` node only has `id` property. In the image below, we have the graph model (schema).

![movies_graph_model](../img/movie-schema.png)

## 4. Connect to Memgraph with GQLAlchemy <a name="connect"></a>

We will use the **GQLAlchemy**'s object graph mapper (OGM) to connect to Memgraph and quickly execute **Cypher** queries. GQLAlchemy also serves as a Python driver/client for Memgraph. We can install it using:

```
pip install gqlalchemy
```

> You may need to install [CMake](https://cmake.org/download/) before installing GQLAlchemy.

In [3]:
from gqlalchemy import Memgraph

In [4]:
memgraph = Memgraph("127.0.0.1", 7687)

Let's make sure that Memgraph is empty before we start with anything else.

In [5]:
memgraph.drop_database()

In [6]:
results = memgraph.execute_and_fetch(
    """
    MATCH (n) RETURN count(n) AS number_of_nodes ;
    """
)
print(next(results))

{'number_of_nodes': 0}


## 5. Define a graph schema <a name="graph-schema"></a>

Next, we have to define the above graph schema in the Python code. This is where **GQLAlchemy** steps in with its Object Graph Mapper.

In [7]:
from typing import Optional
from gqlalchemy import Node, Relationship, Field

class Movie(Node):
    id: int = Field(index=True, unique=True, exists=True, db=memgraph)
    title: Optional[str] = Field()
        

class User(Node):
    id: int = Field(index=True, unique=True, exists=True, db=memgraph)
    name: Optional[str] = Field()
        
class Genre(Node):
    name: str = Field()

class Rated(Relationship, type="RATED"):
    rating: float = Field()
    timestamp: float = Field()

class OfGenre(Relationship, type="OF_GENRE"):
    pass

What’s going on here:

- `Node` is a Python class that maps to a graph object in `Memgraph`.
- Classes that inherit from `Node` map to a single label in the graph database.
- In this case, the class `Movie` maps to the label `:Movie`.
- The properties `id` and `title` are mapped to properties of the nodes labeled `Movie` in the graph database. Their types must be defined and are enforced.
- If the property type is missing the `Optional` keyword, then it can’t be `None` or missing.
- `Field` is a function from pydantic that you can use to define constraints in the graph database like uniqueness constraints, indexes and existence constraints.
- Whenever you provide a constraint to the `Field` function, you have to provide a `Memgraph` object in the `db` argument, so those constraints can be enforced.
- In this instance, uniqueness and exists constraints, as well as index are added to `Memgraph` on the label `Movie` and the property `id`.

## 6. Creating and returning nodes and relationships <a name="create-return"></a>

Before we import the whole dataset from the CSV files, let's see how we can create a simple node and relationship. First, we will create one `User` node and one `Movie` node.

In [8]:
user_node = User(id=1, name="Katarina")
movie_node = Movie(id=1, title="LOTR")

user_node.save(memgraph)
movie_node.save(memgraph)

print(user_node)
print(movie_node)

<User id=0 labels={'User'} properties={'id': 1, 'name': 'Katarina'}>
<Movie id=1 labels={'Movie'} properties={'id': 1, 'title': 'LOTR'}>


Now, let's try to create a node using the Cypher query language. We will create a user with an existing `id` just to check if the uniqueness constraint on the property `id` is set correctly.

In [9]:
try:
    results = memgraph.execute(
        """
        CREATE (:User {name: "Katarina"});
        """
    )
except Exception:
    print("Error: A user with the name Katarina is already in the database.")

Error: A user with the name Katarina is already in the database.


Next, let's create a relationship of type `RATED` between the existing `User` and `Movie` nodes.

In [10]:
from time import time

rated_relationship = Rated(_start_node_id=user_node._id, _end_node_id=movie_node._id, rating=5.0, timestamp=time())

rated_relationship.save(memgraph)

print(rated_relationship)

<Rated id=0 start_node_id=0 end_node_id=1 nodes=(0, 1) type=RATED properties={'rating': 5.0, 'timestamp': 1648401418.0004978}>


## 7. Importing data from CSV files <a name="import"></a>

First, let's clean up the database to import only data from the CSV files.


In [11]:
memgraph.drop_database()

First, we need to download the `movies.csv` and `ratings.csv` files. To import it into Memgraph, we need to copy it to the Docker container where Memgraph is running.
Find the `CONTAINER_ID` by running:

```
docker ps
```

Copy the files with the following commands (don't forget to replace `CONTAINER_ID`):

```
docker cp movies.csv CONTAINER_ID:movies.csv
```

```
docker cp ratings.csv CONTAINER_ID:ratings.csv
```

We will import data to Memgraph using the `LOAD CSV` Cypher clause that enables us to load and use data from a CSV file of our choosing in a row-based manner within a query. Memgraph supports the Excel CSV dialect, the most common one. For more info, check out [Memgraph Docs](https://memgraph.com/docs/memgraph/import-data/load-csv-clause).

The following method will run the `LOAD CSV` clause inside Memgraph and create `Movie` and `Genre` nodes and the `OF_GENRE` relationship between them.

In [12]:
memgraph.execute(
    """
    LOAD CSV FROM "/movies.csv" WITH HEADER AS row
    MERGE (m:Movie {id: toInteger(row.movieId), title: row.title}) 
    WITH m, row
    UNWIND split(row.genres, '|') as genre
    MERGE (m)-[:OF_GENRE]->(:Genre {name: genre});
    """
)

DatabaseError: CSV file not found: /movies.csv

All that is left to create is `User` node and connect it to the `Movie` node that the user rated. We will again use the `LOAD CSV` clause to import data from the `ratings.csv` file.

In [ ]:
memgraph.execute(
    """
    LOAD CSV FROM "/ratings.csv" WITH HEADER AS row
    MATCH (m:Movie {id: toInteger(row.movieId)})
    MERGE (u:User {id: toInteger(row.userId)})
    MERGE (u)-[:RATED {rating: toFloat(row.rating), timestamp: toFloat(row.timestamp)}]->(m);
    """
)

Great! We imported all the data into Memgraph. Feel free to check it out in Memgraph Lab at `localhost:3000`.

## 8. Querying the database and retrieving results <a name="query"></a>

The simplest usage of the **Cypher query language** is to find data stored in the database. For that, you can use one of the following clauses:

- `MATCH` which searches for patterns.
- `WHERE` for filtering the matched data.
- `RETURN` for defining what will be presented to the user in the result set.
- `UNION` and `UNION ALL` for combining results from multiple queries.
- `UNWIND` for unwinding a list of values as individual rows.

Let's create a simple `MATCH` query to find all movies and order them by their title in ascending order.

In [ ]:
results = memgraph.execute_and_fetch(
    """
    MATCH (m:Movie)
    RETURN m
    ORDER BY m.title ASC
    LIMIT 10;
    """
)

In [ ]:
results = list(results)

for result in results:
    print(result["m"])

We may wonder whether these results have been correctly serialized back to the defined schema. We can check that out:

In [ ]:
m = results[8]["m"]

print("Movie: ", m.title)
print("Type: ", type(m))

As expected, the movie in the result is an instance of `Movie` class and it has properties `id` and `title` that can be read.

If you're not keen on Cypher query language, you can try out the GQLAlchemy **query builder**. Let's rewrite the above Cypher query with the help of the query builder:

In [ ]:
from gqlalchemy import match

results_from_qb = (
    match()
    .node(labels="Movie", variable="m")
    .return_()
    .order_by("m.title ASC")
    .limit(10)
    .execute()
)
results_from_qb = list(results_from_qb)

for result in results_from_qb:
    print(result["m"])


As expected, the result is the same as above.

## 9. Example queries  <a name="exercises"></a>


### 🔷 Example 1: Find out how many users there are in the database.

The correct Cypher query would be:

```
MATCH (n:User)
RETURN count(n) AS num_of_users;
```

You can try it out in Memgraph Lab at `localhost:3000`.

With GQLAlchemy's query builder, the solution is:

In [ ]:
total_users = (
    match()
    .node(labels="User", variable="u")
    .return_({"count(u)": "num_of_users"})
    .execute()
)

results = list(total_users)
for result in results:
    print(result["num_of_users"])

#### ❓Exercise 1: Find out how many genres there are in the database.

***

### 🔷 Example 2: Find out how many users rated the movie with the title 'Home Alone (1990)'.

The correct Cypher query would be:

```
MATCH (u:User)-[:RATED]->(:Movie {title: 'Home Alone (1990)'})
RETURN count(u) AS num_of_users;
```

You can try it out in Memgraph Lab at `localhost:3000`.

With GQLAlchemy's query builder, the solution is:

In [ ]:
home_alone = (
    match()
    .node(labels="User", variable="u")
    .to("RATED")
    .node(labels="Movie", variable="m")
    .where("m.title", "=", "Home Alone (1990)")
    .return_({"count(u)": "num_of_users"})
    .execute()
)

results = list(home_alone)

for result in results:
    print(result["num_of_users"])

#### ❓Exercise 2: Find out to how many genres movie 'Matrix, The (1999)' belongs to.

***

### 🔷 Example 3: Find out the name of the genres the movie 'Blue Lagoon, The (1980)' belongs to.

The correct Cypher query would be:

```
MATCH (:Movie {title: 'Blue Lagoon, The (1980)'})-[:OF_GENRE]->(g:Genre)
RETURN g.name AS genre;
```

You can try it out in Memgraph Lab at `localhost:3000`.

With GQLAlchemy's query builder, the solution is:

In [ ]:
genres = (
    match()
    .node(labels="Movie", variable="m")
    .to("OF_GENRE")
    .node(labels="Genre", variable="g")
    .where("m.title", "=", "Blue Lagoon, The (1980)")
    .return_({"g.name": "genre"})
    .execute()
)

results = list(genres)

for result in results:
    print(result["genre"])

#### ❓Exercise 3: Find out the title of the movies that the user with `id` 1 rated.

***

### 🔷 Example 4: List 10 movies of 'Comedy' and 'Action' genres and sort them by title.

The correct Cypher query would be:

```
MATCH (m:Movie)-[:OF_GENRE]->(:Genre {name: "Action"})
MATCH (m)-[:OF_GENRE]->(:Genre {name: "Comedy"})
RETURN m.title
ORDER BY m.title
LIMIT 10;
```

You can try it out in Memgraph Lab at `localhost:3000`.

With GQLAlchemy's query builder, the solution is:

In [ ]:
movies = (
    match()
    .node(labels="Movie", variable="m")
    .to("OF_GENRE")
    .node(labels="Genre", variable="g1")
    .where("g1.name", "=", "Action")
    .match()
    .node(labels="Movie", variable="m")
    .to("OF_GENRE")
    .node(labels="Genre", variable="g2")
    .where("g2.name", "=", "Comedy")
    .return_({"m.title": "movie"})
    .order_by("m.title")
    .limit(10)
    .execute()
)

results = list(movies)

for result in results:
    print(result["movie"])

#### ❓Exercise 4: List 15 movies of 'Documentary' and 'Comedy' genres and sort them by title descending.

***

### 🔷 Example 5: Find out the average rating of the 'Star Wars: Episode IV - A New Hope (1977)' movie.

Here we're going to use one of the **Cypher functions**. We are going to use aggregation `avg` function, that returns an average value of rows with numerical values generated with the `MATCH` or `UNWIND` clause. The list of the supported functions can be found in the [Cypher manual](https://memgraph.com/docs/cypher-manual/functions).

The correct Cypher query would be:

```
MATCH (:User)-[r:RATED]->(:Movie {title: 'Star Wars: Episode IV - A New Hope (1977)'})
RETURN avg(r.rating);
```

You can try it out in Memgraph Lab at `localhost:3000`.

With GQLAlchemy's query builder, the solution is:

In [ ]:
rating = (
    match()
    .node(labels="User")
    .to("RATED", variable="r")
    .node(labels="Movie", variable="m")
    .where("m.title", "=", "Star Wars: Episode IV - A New Hope (1977)")
    .return_({"avg(r.rating)": "average_rating"})
    .execute()
)

results = list(rating)

for result in results:
    print(result["average_rating"])

#### ❓Exercise 5: Find out the minimum rating of the 'Star Wars: Episode I - The Phantom Menace (1999)' movie.

## 10. Want to know more? <a name="extra"></a>

### A. Movie recommender <a name="movie"></a>

We will implement a simple memory based [collaborative filtering](https://en.wikipedia.org/wiki/Collaborative_filtering). The memory-based approach uses user rating data to compute the similarity between users or items.

In [ ]:
ratings = memgraph.execute_and_fetch(
    """
    MATCH (u1:User {id:608})-[r1:RATED]->(m:Movie)<-[r2:RATED]-(u2:User)
    WITH u2.id AS u2_id, avg(abs(r1.rating-r2.rating)) AS similarity, count(*) AS similar_user_count
    WHERE similar_user_count > 2
    WITH u2_id
    ORDER BY similarity
    LIMIT 10
    WITH collect(u2_id) AS similar_user_set
    MATCH (m2: Movie)<-[r3:RATED]-(u3:User)
    WHERE u3.id IN similar_user_set
    WITH m2, avg(r3.rating) AS prediction_rating
    RETURN m2.title AS title, prediction_rating
    ORDER BY prediction_rating DESC;
    """
)

results = list(ratings)

for result in results:
    print(result["title"], result["prediction_rating"])

**How does this query work?**

This query has two parts:

1. Finding similar users
2. Predicting the score for some movie (recommendation)

In the first part, we are looking for **similar users**. Two users are considered similar if they tend to give similar ratings to the same movies. For the target user (with `id` 608) and some other user we are searching for the same movies:

```
MATCH (u1:User {id:608})-[r1:RATED]->(m:Movie)<-[r2:RATED]-(u2:User)
```

But that is not enough to find similar users. We need to choose users with similar ratings:

```
WITH u2.id AS u2_id,
     avg(abs(r1.rating-r2.rating)) AS similarity,
     count(*) AS similar_user_count
WHERE similar_user_count > 2
WITH u2_id
ORDER BY similarity
LIMIT 10
WITH collect(u2_id) AS similar_user_set
```

The similarity is being calculated as the average distance between target user rating and some other user rating on the same set of movies. The `similar_user_count` limit is used for filtering users who have at least 2 movies in common with the target user. The `similar_user_set` size is used to peek top 10 similar users (10 or less).

We have a similar user set so we can calculate the prediction rating for all movies in the database:

```
MATCH (m2: Movie)<-[r3:RATED]-(u3:User)
WHERE u3.id IN similar_user_set
WITH m2, avg(r3.rating) AS prediction_rating
RETURN m2.title AS title, prediction_rating
ORDER BY prediction_rating DESC;
```


We encourage you to play with some parameters, like `similar_user_count` limit and `similar_user_set` size limit. You can also try to use different similarity functions, for example [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance). Then we would calculate the `similarity` like this:

```
sqrt(reduce(a=0, x IN collect((r1.rating - r2.rating) * (r1.rating - r2.rating)) | a + x)) AS similarity
```

### B. Graph analytics <a name="graph-analytics"></a>

We will perform the **PageRank** algorithm on our database and find out which movie is the most popular. Today, the most popular search engine in the world, Google, owes its popularity solely to this algorithm, developed in the early days by its founders. The PageRank algorithm outputs probability distribution used to represent the likelihood that a person randomly clicking on links will arrive at any particular page.

We can calculate ranks using the Cypher query language:

```
CALL pagerank.get()
YIELD *
WITH node, rank
WHERE node:Movie
RETURN node.title AS movie_title, rank
ORDER BY rank DESC
LIMIT 50;
```

With GQLAlchemy's query builder, we have:

In [ ]:
from gqlalchemy import call
results = list(
            call("pagerank.get")
            .yield_()
            .with_({"node": "node", "rank": "rank"})
            .add_custom_cypher("WHERE node:Movie")
            .return_({"node.title": "movie_title", "rank": "rank"})
            .order_by("rank DESC")
            .limit(50)
            .execute()
        )

for result in results:
    print(result["movie_title"], result["rank"])

That's it! Hopefully, you gained new knowledge and enjoyed this workshop. If you have any questions, don't hesitate to contact me directly or [join our Discord server](https://discord.gg/memgraph).

> The solutions to all exercises are located in the [gqlalchemy-solutions notebook](https://github.com/pyladiesams/graphdbs-gqlalchemy-beginner-mar2022/blob/master/solutions/gqlalchemy-solutions.ipynb). The best way to learn is to solve the exercises yourself and then check out the answers.

<h3 align="center"><a href=#toc>⬆️ GO TO TOP ⬆️</a></h3> 